In [1]:
import tweepy
import pandas as pd
import time 
import csv
import numpy as np
from pathlib import Path

In [2]:
# create client tweepy
bearer_token = "AAAAAAAAAAAAAAAAAAAAAHqVaQEAAAAAQeqIjAve3PYwMwqZagLUY0MTapI%3D9qBwaEmiTtt1uWBX6Nnrnz71XibxrwsUHdEpZbzlaO4laVxeiI"
client = tweepy.Client(bearer_token)

In [4]:
fileCsv = Path("data/test-ebt.csv")
if fileCsv.is_file():
    df = pd.read_csv('data/test-ebt.csv')
    df = df.drop(columns=['Unnamed: 0'])
else:
    column_names = ["tweet_id","author_id","author_name","author_username","author_location","content","content_clean","like_count","retweet_count","reply_count","source","keyword","address","longitude","latitude","province","city","country","created_at"]
    df = pd.DataFrame(columns = column_names)
    df

In [13]:
keyword = "ebt -is:retweet lang:id"
search_df = pd.DataFrame()
loop = 1
loop_conditional = False
token = None
start_time = '2012-01-01T00:00:00Z'
end_time = '2022-06-30T00:00:00Z'

# initial search to get next_token or pagination
search = client.search_recent_tweets(
    query = keyword,
    tweet_fields=['public_metrics', 'author_id', 'created_at', 'source', 'text', 'geo'], 
    user_fields = ['name','username','location','verified'],
    expansions = ['geo.place_id', 'author_id'],
    place_fields = ['country','country_code'],
    # start_time=start_time,
    # end_time=end_time,
    max_results=10, 
    # next_token=token,
)

In [6]:
# save result of search to dataframe
for row in search.data:
    new_array = {}
    new_array["tweet_id"] = row['id'] 
    new_array["author_id"] = row['author_id'] 
    new_array["author_name"] = "" 
    new_array["author_username"] = "" 
    new_array["author_location"] = "" 
    new_array["content"] = row['text'] 
    new_array['content_clean'] = ""
    new_array["like_count"] = row['public_metrics']['like_count'] 
    new_array["retweet_count"] = row['public_metrics']['retweet_count'] 
    new_array["reply_count"] = row['public_metrics']['reply_count'] 
    new_array["source"] = row['source'] 
    new_array["keyword"] = "ebt" 
    new_array["address"] = "" 
    new_array["longitude"] = ""
    new_array["latitude"] = ""
    new_array["province"] = ""
    new_array["city"] = ""
    new_array["country"] = ""
    new_array["created_at"] = row['created_at'] 
    temp_data = pd.json_normalize(new_array ,  sep = "_")
    search_df = pd.concat([search_df, temp_data], ignore_index=True)  
    loop+=1

In [7]:
for row in search.includes['users']:
    search_df.loc[search_df["author_id"] == row.id, "author_username"] = row.username
    search_df.loc[search_df["author_id"] == row.id, "author_name"] = row.name
    search_df.loc[search_df["author_id"] == row.id, "author_location"] = row.location

In [8]:
df = df.append(search_df, ignore_index=True)

C:\Users\asus\AppData\Local\Temp\ipykernel_25404\652426727.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(search_df, ignore_index=True)


In [9]:
df = df.drop_duplicates(subset=['tweet_id'])

In [10]:
df.head(5)

,tweet_id,author_id,author_name,author_username,author_location,content,content_clean,like_count,retweet_count,reply_count,source,keyword,address,longitude,latitude,province,city,country,created_at
0,1562112713303552000,47596019,LIPUTAN6,liputan6dotcom,INDONESIA,"EBT Belum Optimal, Produsen Migas Soroti Ketah...",,0,0,0,dlvr.it,energi baru terbarukan,,,,,,,2022-08-23 16:21:08+00:00
1,1562073423840530435,791948030,Dunia Energi,DuniaEnergi,"Jakarta Capital Region, Indone","Perkembangan EBT Lambat, Pemerintah Diminta Fo...",,0,0,0,WordPress.com,energi baru terbarukan,,,,,,,2022-08-23 13:45:00+00:00
2,1562071969905790978,1361931563965943808,ruangenerginews,ruangenerginews,"Jakarta, Indoesia","Jakarta, https://t.co/KLJ7NohX9r - Lebih lanj...",,3,0,0,Twitter Web App,energi baru terbarukan,,,,,,,2022-08-23 13:39:14+00:00
3,1562066328592080896,416737198,@bianconero,_il_principino,221b Baker Street,"Kepikiran juga sih, orang listrik aja msh disu...",,0,0,0,Twitter for Android,energi baru terbarukan,,,,,,,2022-08-23 13:16:49+00:00
4,1562056711195635712,1380017096390676483,FORTUNE Indonesia,Fortune_IDN,None,Harga tinggi batu bara diprediksi tidak bertah...,,0,0,0,dlvr.it,energi baru terbarukan,,,,,,,2022-08-23 12:38:36+00:00


In [11]:
df.to_csv('data/test-ebt.csv')